# Test point-based registration

In [ ]:
%matplotlib widget
import numpy as np
import os
import open3d as o3d
import copy
import time

import planeslam.io as io

Read in airsim LiDAR and pose data

In [ ]:
# Read in point cloud data
binpath = os.path.join(os.getcwd(),'..', 'data', 'airsim', 'blocks_20_samples_1', 'lidar', 'Drone0')
PC_data = io.read_lidar_bin(binpath)

In [ ]:
# Read in ground-truth poses (in drone local frame)
posepath = os.path.join(os.getcwd(),'..', 'data', 'airsim', 'blocks_20_samples_1', 'poses', 'Drone0')
drone_positions, drone_orientations = io.read_poses(posepath)

Open3D ICP registration

In [ ]:
# Visualization helper
def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp],
                                      zoom=0.4459,
                                      front=[0.9288, -0.2951, -0.2242],
                                      lookat=[1.6784, 2.0612, 1.4451],
                                      up=[-0.3402, -0.9189, -0.1996])

In [ ]:
# Initialize source and target
source = o3d.geometry.PointCloud()
source.points = o3d.utility.Vector3dVector(PC_data[0])
source.estimate_normals()
source.orient_normals_towards_camera_location()

target = o3d.geometry.PointCloud()
target.points = o3d.utility.Vector3dVector(PC_data[1])
target.estimate_normals()
target.orient_normals_towards_camera_location()

In [ ]:
# Visualize
o3d.visualization.draw_geometries([target])

In [ ]:
# Initial guess transform 
trans_init = np.eye(4)
threshold = 0.02

evaluation = o3d.pipelines.registration.evaluate_registration(
    source, target, threshold, trans_init)
print(evaluation)

In [ ]:
# Point-to-point ICP
print("Apply point-to-point ICP")
start_time = time.time()
reg_p2p = o3d.pipelines.registration.registration_icp(
    source, target, threshold, trans_init,
    o3d.pipelines.registration.TransformationEstimationPointToPoint())
print("Elapased time: ", time.time() - start_time)
print(reg_p2p)
print("Transformation is:")
print(reg_p2p.transformation)
draw_registration_result(source, target, reg_p2p.transformation)

In [ ]:
demo_icp_pcds = o3d.data.DemoICPPointClouds()
source = o3d.io.read_point_cloud(demo_icp_pcds.paths[0])
target = o3d.io.read_point_cloud(demo_icp_pcds.paths[1])

np.asarray(source.normals)

In [ ]:
print("Apply point-to-plane ICP")
start_time = time.time()
reg_p2l = o3d.pipelines.registration.registration_icp(
    source, target, threshold, trans_init,
    o3d.pipelines.registration.TransformationEstimationPointToPlane())
print("Elapased time: ", time.time() - start_time)
print(reg_p2l)
print("Transformation is:")
print(reg_p2l.transformation)
draw_registration_result(source, target, reg_p2l.transformation)